In [ ]:
# data manipulation libraries
import numpy as np
import pandas as pd
from ast import literal_eval
from sklearn.model_selection import train_test_split

# pyspark
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.sql import types, functions as F

# random seed for reproducibility
RANDOM_SEED = 15

In [ ]:
app_name = 'reseller_pilot_sampling'

conf = (SparkConf()
         .setMaster('yarn-client')
         .setAppName(app_name)
         .set("spark.yarn.queue", "root.hue_dmp")
         .set("spark.executor.memory", "16G")
         .set("spark.executor.cores","4")
         .set("spark.driver.memory", "16G")
         .set("spark.default.parallelism", "8")
         .set("spark.sql.shuffle.partitions", "1000")
         .set("spark.shuffle.service.enabled", "true")
         .set("spark.dynamicAllocation.enabled", "true")
         .set("spark.dynamicAllocation.minExecutors", "1")
         .set("spark.dynamicAllocation.maxExecutors", "20")
         .set("spark.dynamicAllocation.initialExecutors", "1")
         .set("spark.yarn.maxAppAttempts", "2")
         .set("spark.sql.parquet.compression.codec", "snappy")
         .set("spark.sql.parquet.binaryAsString", "true")
         .set("spark.driver.memoryOverhead", "4096")
         .set("spark.yarn.driver.memoryOverhead", "4096")
         .set("spark.yarn.executor.memoryOverhead", "4096")
         .set("spark.executor.heartbeatInterval", "20s")
         .set("spark.network.timeout", "800s")
         .set("spark.sql.broadcastTimeout", "1200")
         .set("spark.sql.hive.convertMetastoreParquet", "false")
         .set("yarn.nodemanager.vmem-check-enabled", "false")
         .set("spark.default.parallelism", "8")
         .set("spark.sql.shuffle.partitions", "1000")
         .set("spark.driver.memory", "16G")
         .set("spark.testing.memory", "2147480000")
         .set("spark.sql.hive.verifyPartitionPath", "false")
         .set("spark.driver.maxResultSize", "0")
         .set("spark.sql.autoBroadcastJoinThreshold", 400*1024*1024)
         .set("yarn.nodemanager.vmem-check-enabled","false")
         .set("spark.hadoop.fs.permissions.umask-mode","002")
)

sc = SparkContext.getOrCreate(conf=conf)
spark = SparkSession(sc)

## Load performance percentiles for outlet in March 2020

In [ ]:
csv_filepath = '/home/cdsw/project_mck_dmp/data/reseller/07_model_output/kfold/performance/ra_mck_int_kfold_performance_percentiles_march2020.csv'
march_perf = pd.read_csv(csv_filepath).set_index('outlet_id')
print(march_perf.shape)
march_perf.head()

In [ ]:
march_perf.training_ground_truths_list_length.describe()

## Join with master table to filter to get region, pv columns

In [ ]:
cols = ['outlet_id', 'fea_region', 'fea_outlets_pv_red_sum_nominal_by_rs_msisdn_sum']
csv_filepath = '/home/cdsw/project_mck_dmp/data/reseller/05_model_input/master_table/ra_master_table_mar2020.csv'
master_table = pd.read_csv(csv_filepath)[cols].set_index('outlet_id')
print(master_table.shape)
master_table.head()

In [ ]:
# join with performance percentile df
march_perf = march_perf.join(master_table, how='left')
march_perf.head()

## Fillna and remove outlets with physical voucher transactions 

In [ ]:
march_perf = march_perf.fillna(value={'fea_outlets_pv_red_sum_nominal_by_rs_msisdn_sum': 0.0})
print(march_perf.shape)
march_perf = march_perf[march_perf.fea_outlets_pv_red_sum_nominal_by_rs_msisdn_sum == 0.0]
print(march_perf.shape)
march_perf.head()

## Load May 2020 master table to get cashflow to calculate deviation %

In [ ]:
cols = ['outlet_id', 'fea_outlet_decimal_total_cashflow_mkios_pv_mean', 'fea_outlets_pv_red_sum_nominal_by_rs_msisdn_sum']
csv_filepath = '/home/cdsw/project_mck_dmp/data/reseller/05_model_input/master_table/ra_master_table_may2020.csv'
may_cashflows = pd.read_csv(csv_filepath)[cols].set_index('outlet_id')
may_cashflows = may_cashflows.fillna({'fea_outlet_decimal_total_cashflow_mkios_pv_mean': 0.0, 'fea_outlets_pv_red_sum_nominal_by_rs_msisdn_sum': 0.0}).rename({'fea_outlet_decimal_total_cashflow_mkios_pv_mean': 'may_2020_cashflow_mkios_pv_mean', 'fea_outlets_pv_red_sum_nominal_by_rs_msisdn_sum': 'physical_voucher_sum_may2020'}, axis=1)
print(may_cashflows.shape)
may_cashflows.head()

In [ ]:
# join with march_perf
march_perf = march_perf.join(may_cashflows, how='left')
march_perf.head()

In [ ]:
# filter out outlets with physical voucher transactions in May 2020
print(march_perf.shape)
march_perf = march_perf[march_perf.physical_voucher_sum_may2020 == 0.0]
print(march_perf.shape)
march_perf.head()

In [ ]:
# calculate percentage deviation
march_perf['cashflow_percentage_change_absolute'] = ((march_perf['may_2020_cashflow_mkios_pv_mean'] - march_perf['ground_truth']) * 100 / march_perf['ground_truth']).abs()
print(march_perf.shape)
march_perf.head()

In [ ]:
march_perf = march_perf[(((march_perf['ground_truth'] >= 400000) & (march_perf['ground_truth'] <= 600000)) | ((march_perf['ground_truth'] >= 240000) & (march_perf['ground_truth'] <= 360000)) | ((march_perf['ground_truth'] >= 80000) & (march_perf['ground_truth'] <= 120000))) & (march_perf['cashflow_percentage_change_absolute'] <= 10)]
print(march_perf.shape)
march_perf.head()

## Calculate percentile cashflows and corresponding differences with actual

In [ ]:
march_perf['training_ground_truths_list'] = march_perf['training_ground_truths_list'].map(literal_eval)

# calculate percentiles cashflows
march_perf['75th_percentile_cashflow'] = march_perf['training_ground_truths_list'].map(lambda lst: np.quantile(lst, 0.75))
march_perf['90th_percentile_cashflow'] = march_perf['training_ground_truths_list'].map(lambda lst: np.quantile(lst, 0.9))
march_perf['95th_percentile_cashflow'] = march_perf['training_ground_truths_list'].map(lambda lst: np.quantile(lst, 0.95))
march_perf['100th_percentile_cashflow'] = march_perf['training_ground_truths_list'].map(lambda lst: np.quantile(lst, 1.0))

# obtain percentile differences against actual
march_perf['diff_with_75th_percentile_cashflow'] = march_perf['75th_percentile_cashflow'] - march_perf['ground_truth']
march_perf['diff_with_90th_percentile_cashflow'] = march_perf['90th_percentile_cashflow'] - march_perf['ground_truth']
march_perf['diff_with_95th_percentile_cashflow'] = march_perf['95th_percentile_cashflow'] - march_perf['ground_truth']
march_perf['diff_with_100th_percentile_cashflow'] = march_perf['100th_percentile_cashflow'] - march_perf['ground_truth']

march_perf.head()

In [ ]:
# temp, split into tertiles
group_values = [100000, 300000, 500000]
subgroups_df = pd.DataFrame(columns=march_perf.columns.tolist() + ['cashflow_group', 'tertile'])

for idx, val in enumerate(group_values):
    if idx == 0:
        increment = 20000
    elif idx == 1:
        increment = 60000
    else:
        increment = 100000
        
    lower_bound, upper_bound = val - increment, val + increment
    filtered_gt_df = march_perf.loc[(march_perf['ground_truth'] >= lower_bound) & (march_perf['ground_truth'] <= upper_bound), :]
    
    # assign new columns
    filtered_gt_df['cashflow_group'] = val
    filtered_gt_df['tertile'] = pd.qcut(filtered_gt_df['diff_with_100th_percentile_cashflow'], 3, labels=['low', 'med', 'high'])
    
    subgroups_df = subgroups_df.append(filtered_gt_df)
    
subgroups_df.index.name = 'outlet_id'
    
print(subgroups_df.shape)
subgroups_df.head()

In [ ]:
subgroups_df[subgroups_df.cashflow_group == 500000].tertile.value_counts()

## Get relevant columns for outlet

In [ ]:
outlet_ids = subgroups_df.index.tolist()
outlet_ids[:5]

In [ ]:
cols = ['outlet_id', 'area', 'regional', 'branch', 'kabupaten', 'kecamatan', 'kelurahan','klasifikasi', 'tipe_outlet'] 
details = spark.read.table('dmp_remote.outlet_dim_dd') \
    .filter((F.col('outlet_id').isin(outlet_ids)) & (F.col('load_date').between('2020-03-01', '2020-03-31'))) \
    .select(cols).distinct().toPandas().set_index('outlet_id')

details.index = details.index.astype(int)
details

In [ ]:
# join with outlet_ids
print(subgroups_df.shape)
joined = subgroups_df.join(details, how='left')
print(joined.shape)
joined.head()

In [ ]:
# drops rows with duplicates
joined = joined.loc[~joined.index.duplicated(), :]
print(joined.shape)
joined.head()

In [ ]:
q_values_cols = [f'q{num}_threshold' for num in range(1,5)]
joined['quartile'] = pd.cut(joined.performance_percentile, bins=[0.0, 25.0, 50.0, 75.0, 100.0], labels=['q1','q2','q3','q4'])
joined[q_values_cols] = joined.training_ground_truths_list.apply(lambda lst: pd.Series({f"q{num}_value": round(np.quantile(lst, num*0.25),2) for num in range(1,5)}))
joined

In [ ]:
sort_by = [
    'cashflow_group',
    'tertile',
    'diff_with_100th_percentile_threshold'
]

cols = [
    'outlet_id',
    'ground_truth',
    'performance_percentile',
    'quartile',
    'q1_threshold',
    'q2_threshold',
    'q3_threshold',
    'q4_threshold',
    '90th_percentile_cashflow',
    '95th_percentile_cashflow',
    'diff_with_75th_percentile_cashflow',
    'diff_with_90th_percentile_cashflow',
    'diff_with_95th_percentile_cashflow',
    'diff_with_100th_percentile_cashflow',
    'area',
    'regional',
    'branch',
    'kabupaten',
    'kecamatan',
    'kelurahan',
    'klasifikasi',
    'tipe_outlet',
    'cashflow_group',
    'tertile',
]

selected = joined.reset_index()[cols].rename(columns={
    'diff_with_75th_percentile_cashflow': 'diff_with_75th_percentile_threshold',
    'diff_with_90th_percentile_cashflow': 'diff_with_90th_percentile_threshold',
    'diff_with_95th_percentile_cashflow': 'diff_with_95th_percentile_threshold',
    'diff_with_100th_percentile_cashflow': 'diff_with_100th_percentile_threshold',
    '90th_percentile_cashflow': '90th_percentile_threshold',
    '95th_percentile_cashflow': '95th_percentile_threshold'}).sort_values(
        by=sort_by
    )

# round to 2 dp
selected = selected.round(2)

# sort values
selected = selected.sort_values(['cashflow_group', 'tertile', 'diff_with_100th_percentile_threshold', 'performance_percentile'])

print(selected.shape)
selected.head()

In [ ]:
selected.tail()

In [ ]:
# save output
selected.to_csv('/home/cdsw/outlets_for_pilots_full_set_expanded.csv', index=False)

## Next steps:
1. Check number of zero cashflow days in May 2020 do not exceed by 5
2. Check outlets' month total cashflow for March 2020 and May 2020 do not deviate by more than 15%
2. Split the pool of outlets into groups and manually balance them to ensure similar distribution of region and total cashflow
    - using High and Low tertile for each cashflow group